This convolutional network (CN) recognises faces from a database of 244 images of 16 individuals. The images are low in quality and variability. The aim of using the chosen data was to see how CN's respond to low quality data, compared to easily classifiable high-quality  data such as the MNIST data set.

The model is successful at matching a face to an individual ~80% of time. Given the limitations in the data set this is still quite impressive. The comparison in results here and in the MNIST model demonstrate how important data quality is.

In [1]:
# Deep Learning CNN model to recognize face
# This script uses an existing database of images and creates a CNN to recognise faces in the images.
# Data extracted from Farukh Hashmi's blog

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)

# IMAGE PRE-PROCESSING for TRAINING and TESTING data

# Specifying the location of the training data
from google.colab import drive
import time
drive.mount('/content/drive')
TrainingImagePath = '/content/drive/My Drive/FaceImages/TrainingImages'


# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True
        )
# Defining pre-processing transformations on raw images of validation data
test_datagen = ImageDataGenerator()


# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical'
)
# Generating the Validation Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical'
)

# Normalising the data.
# training_set = tf.keras.utils.normalize(training_set, axis = 1)
# test_set = tf.keras.utils.normalize(test_set, axis = 1)

# Printing class labels for each face
test_set.class_indices
print("Mapping of Face and its ID", test_set.class_indices)

# The number of neurons in the output layer is the number of people in the data
OutputNeurons = len(test_set.class_indices)
print('\n The Number of output neurons: ', OutputNeurons)


#-----------------------------------------------------------------------------#
# Creating the CNN architechture

input_shape = (64, 64, 3)
inputs = tf.keras.layers.Input(shape=input_shape)

##### Convolutional layers #####
layer = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu)(inputs)
layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)

layer = tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu)(layer)
layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(layer)
##### --------------------- #####

##### Flattening layer #####
layer = tf.keras.layers.Flatten()(layer)
##### ---------------- #####

##### Fully connected layer #####
layer = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(layer)
##### ----------------------- #####

##### Output layer #####
outputs = tf.keras.layers.Dense(units=OutputNeurons, activation=tf.nn.softmax)(layer)
##### ------------ #####


# Compile the CNN
model = tf.keras.Model(inputs, outputs)  # Initialize the CNN
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

StartTime = time.time()

model.fit(
    training_set,
    steps_per_epoch=len(training_set),  # Adjusted based on the number of batches per epoch
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set)
)
EndTime = time.time()

print("###### Total Time Taken: ", round((EndTime - StartTime) / 60), 'Minutes ######')

Mounted at /content/drive
Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.
Mapping of Face and its ID {'face1': 0, 'face10': 1, 'face11': 2, 'face12': 3, 'face13': 4, 'face14': 5, 'face15': 6, 'face16': 7, 'face2': 8, 'face3': 9, 'face4': 10, 'face5': 11, 'face6': 12, 'face7': 13, 'face8': 14, 'face9': 15}

 The Number of output neurons:  16


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 60, 60, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 26, 26, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 10816)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         692,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           1,040 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 747,024 (2.85 MB)

 Trainable params: 747,024 (2.85 MB)

...

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 62s 537ms/step - accuracy: 0.0654 - loss: 67.5777 - val_accuracy: 0.0902 - val_loss: 3.7819
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 483ms/step - accuracy: 0.1259 - loss: 3.0441 - val_accuracy: 0.1148 - val_loss: 2.6606
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 690ms/step - accuracy: 0.1728 - loss: 2.6270 - val_accuracy: 0.3689 - val_loss: 2.0852
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 457ms/step - accuracy: 0.3624 - loss: 2.0565 - val_accuracy: 0.7336 - val_loss: 1.1424
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 464ms/step - accuracy: 0.6533 - loss: 1.2802 - val_accuracy: 0.7828 - val_loss: 0.7175
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
###### Total Time Taken:  2 Minutes ######


The code cell below runs the model *num_runs* times, to find an average accuracy. The inherently stochastic process of neural network modelling is much more noticable for smaller data sets - some runs yield a 90% accuracy, whereas some runs only yield a 65% accuracy. **WARNING:** will take **2 minutes × num_runs** to run!

In [4]:
tf.random.set_seed(42)
np.random.seed(42)


num_runs = 5
print(f"Warning! Expect this code cell to take {num_runs * 2} minutes to execute!")
for i in range(1, num_runs + 1):
    print(f"\n----- Run {i} -----")

    # Create model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu,
                               input_shape=(64, 64, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation=tf.nn.relu),
        tf.keras.layers.Dense(units=OutputNeurons, activation=tf.nn.softmax)
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        training_set,
        steps_per_epoch=len(training_set),
        epochs=10,
        validation_data=test_set,
        validation_steps=len(test_set),
        verbose=0  # Set verbose to 0 to suppress training output
    )

    # Evaluate the model on test set
    _, accuracy = model.evaluate(test_set, verbose=0)

    print(f"Accuracy for Run {i}: {accuracy}")

Warning! Expect this code cell to take 10 minutes to execute!

----- Run 1 -----


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Accuracy for Run 1: 0.8237704634666443

----- Run 2 -----
Accuracy for Run 2: 0.7459016442298889

----- Run 3 -----
Accuracy for Run 3: 0.8483606576919556

----- Run 4 -----
Accuracy for Run 4: 0.8360655903816223

----- Run 5 -----
Accuracy for Run 5: 0.4836065471172333


Now the CNN is trained with the data set predictions can be made. The cell below extracts a single testing image and resizes it to the model requirements. The resized image is given as input to the model, which returns an output **result**.

Some basic data extraction is done to find the ID of the real image and of the predicted image. Finally a plot of the input titled with it's real and predicted class is given.

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

# Define image paths for all images
image_paths = [
    '/content/drive/My Drive/FaceImages/TestingImages/face10/1face10.jpg',
    '/content/drive/My Drive/FaceImages/TestingImages/face14/2face14.jpg',
    '/content/drive/My Drive/FaceImages/TestingImages/face6/1face6.jpg',
    '/content/drive/My Drive/FaceImages/TestingImages/face1/3face1.jpg',
    '/content/drive/My Drive/FaceImages/TestingImages/face2/1face2.jpg',
    '/content/drive/My Drive/FaceImages/TestingImages/face13/1face13.jpg'
]

plt.figure(figsize=(12, 8.5))

# Iterate over each image path
for i, image_path in enumerate(image_paths, 1):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    result = model.predict(test_image, verbose=0)  # Using the trained model for prediction

    # Manual output based on the class indices
    class_indices = {'face1': 0, 'face10': 1, 'face11': 2, 'face12': 3, 'face13': 4, 'face14': 5, 'face15': 6, 'face16': 7, 'face2': 8, 'face3': 9, 'face4': 10, 'face5': 11, 'face6': 12, 'face7': 13, 'face8': 14, 'face9': 15}

    # Extracting the actual face number from the file name
    file_name = image_path.split('/')[-1]  # Extracting the file name from the path
    actual_face_number = int(file_name.split('face')[1].split('.')[0])  # Extracting the face number from the file name

    # Manual output based on the manual class indices
    predicted_class_index = np.argmax(result)  # Get the index of the predicted class
    predicted_class = [k for k, v in class_indices.items() if v == predicted_class_index][0]  # Get the predicted class label

    # Extracting only the numeric part from the predicted class
    predicted_class_numeric = int(predicted_class.split('face')[1])  # Extracting the numeric part from the predicted class

    # Plot the images
    plt.subplot(2, 3, i)
    plt.imshow(image.load_img(image_path))  # Plot the image
    plt.title(f'Actual Face Number: {actual_face_number}\nPredicted Class: {predicted_class_numeric}')  # Set the title with the actual face number and predicted class
    plt.axis('off')

plt.show()


In [ ]:
import os
import pandas as pd

# Define the path to the testing images folder
testing_images_folder = '/content/drive/My Drive/FaceImages/TestingImages/'

# Initialize an empty DataFrame to store the predictions
predictions_df = pd.DataFrame(columns=['Actual Class'] + [f'Image {i}' for i in range(1, 5)])

# Iterate over each class folder in the testing images folder
for class_folder in os.listdir(testing_images_folder):
    class_path = os.path.join(testing_images_folder, class_folder)
    if os.path.isdir(class_path):
        # Get the class label (e.g., 'face1', 'face2', etc.)
        class_label = class_folder.split('face')[1]

        # Initialize a list to store the predictions for each image in the class
        class_predictions = [f'face{class_label}']

        # Iterate over each image in the class folder
        for i in range(1, 5):
            image_path = os.path.join(class_path, f'{i}face{class_label}.jpg')
            if os.path.isfile(image_path):
                # Load and preprocess the image
                test_image = image.load_img(image_path, target_size=(64, 64))
                test_image = image.img_to_array(test_image)
                test_image = np.expand_dims(test_image, axis=0)

                # Predict the class label using the trained model
                result = model.predict(test_image, verbose=0)
                predicted_class_index = np.argmax(result)
                predicted_class_label = [k for k, v in class_indices.items() if v == predicted_class_index][0]

                # Compare the predicted class with the actual class
                if predicted_class_label == class_folder:
                    class_predictions.append(1)  # Correct prediction
                else:
                    class_predictions.append(0)  # Incorrect prediction
            else:
                # Image file does not exist
                class_predictions.append(None)

        # Append the class predictions to the DataFrame
        predictions_df.loc[len(predictions_df)] = class_predictions

# Display the predictions DataFrame
print(predictions_df)



In [ ]:
# Calculate the percentage accuracy
accuracy = (predictions_df.iloc[:, 1:] == 1).mean().mean() * 100

print(f"Percentage Accuracy: {accuracy:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model
import os
import sys

# Redirect stdout to suppress verbose output
sys.stdout = open(os.devnull, 'w')

# Define intermediate models to extract activations from specific layers
conv1_output_model = Model(inputs=model.input, outputs=model.layers[2].output)

# List of image paths
image_paths = [
    '/content/drive/My Drive/FaceImages/TestingImages/face10/1face10.jpg',
    # Add more image paths as needed
]

#    '/content/drive/My Drive/FaceImages/TestingImages/face12/1face12.jpg',
#    '/content/drive/My Drive/FaceImages/TestingImages/face13/1face13.jpg',

# Iterate over each image path
for idx, image_path in enumerate(image_paths, 1):
    # Load the image and preprocess it
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    # Get activations from the first convolutional layer
    conv1_output = conv1_output_model.predict(test_image)

    # Plot the output of the first convolutional layer as a single image
    plt.figure(figsize=(6, 6))
    plt.imshow(conv1_output[0, :, :, 13], cmap='viridis')  # Display the output of the first channel
    #plt.title(f'Activations for Image {idx}')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

# Restore stdout
sys.stdout = sys.__stdout__


In [ ]:
activation_model = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)   #[1] represents the first conv layer. Try changing to [2] to see the output of the 2nd Conv2d layer.


image_index = 1                                     #Change image_index for a different image
image = x_test[image_index].reshape(1, 28, 28, 1)

activations = activation_model.predict(image)

#Plot the activations for the first image                          #This creates a lattice of the 64 outputs for the image.
plt.figure(figsize=(8, 8))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.imshow(activations[0, :, :, i], cmap='viridis')
    plt.axis('off')
plt.show()

# plt.imshow(activations[0, :, :, 1], cmap='viridis')               #This shows just one image from the lattice above. Change 1 to any value 0<=x<63.
# plt.axis('off')

In [ ]:
from tensorflow.keras.models import Model

# Define intermediate models to extract activations from specific layers
conv1_output_model = Model(inputs=model.input, outputs=model.layers[5].output)
conv2_output_model = Model(inputs=model.input, outputs=model.layers[2].output)

# Load the image and preprocess it
test_image = image.load_img(image_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Get activations from each convolutional layer
conv1_output = conv1_output_model.predict(test_image)
conv2_output = conv2_output_model.predict(test_image)

# Plot the original image, output of the first convolutional layer, and output of the second convolutional layer
plt.figure(figsize=(15, 5))

# Original Image
plt.subplot(1, 3, 1)
plt.imshow(image.load_img(image_path))
plt.title('Original Image')
plt.axis('off')

# Output of the first convolutional layer
plt.subplot(1, 3, 2)
plt.imshow(conv1_output[0, :, :, 1])  # Plot the first channel of the output
plt.title('The Convolutional Layer')
plt.axis('off')

# Output of the second convolutional layer
plt.subplot(1, 3, 3)
plt.imshow(conv2_output[0, :, :, 1])  # Plot the first channel of the output
plt.title('The Pooling Layer')
plt.axis('off')

plt.show()
